<a href="https://colab.research.google.com/github/VanessaTrajano/CampoMinado-Python/blob/main/CampoMinado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import re

class Tabuleiro:
  def __init__(self, tamanhoTabuleiro, numeroBombas):
    self.tamanhoTabuleiro = tamanhoTabuleiro
    self.numeroBombas = numeroBombas

    self.tabuleiro = self.criaNovoTabuleiro()
    self.atribuiValoresAoTabuleiro()

    self.escavado = set()

  def criaNovoTabuleiro(self):
    tabuleiro = [[None for _ in range(self.tamanhoTabuleiro)] for _ in range(self.tamanhoTabuleiro)]

    bombasPlantadas = 0
    while bombasPlantadas < self.numeroBombas:
      local = random.randint(0, self.tamanhoTabuleiro**2 - 1)
      linha = local // self.tamanhoTabuleiro
      coluna = local % self.tamanhoTabuleiro

      if tabuleiro[linha][coluna] == "*":
        # isso quer dizer que já existe uma bomba na posição
        continue

      tabuleiro[linha][coluna] = "*" # planta uma bomba
      bombasPlantadas += 1

    return tabuleiro

  def atribuiValoresAoTabuleiro(self):
    # aqui calculamos o número das posições
    for l in range(self.tamanhoTabuleiro):
      for c in range(self.tamanhoTabuleiro):
        if self.tabuleiro[l][c] == "*":
          # se já é uma bomba, não queremos calcular nada
          continue

        self.tabuleiro[l][c] = self.pegaNumeroDeBombasVizinhas(l, c)

  def pegaNumeroDeBombasVizinhas(self, linha, coluna):
    # temos que conferir todas as casas vizinhas para contar quantas bombas tem
    # acima à esquerda (linha-1, coluna-1)
    # acima no meio (linha-1, coluna)
    # acima à direita (linha-1, coluna+1)
    # esquerda (linha, coluna-1)
    # direita (linha, coluna+1)
    # abaixo à esquerda (linha+1, coluna-1)
    # abaixo no meio (linha+1, coluna)
    # abaixo à direita (linha+1, coluna+1)

    numeroDeBombasVizinhas = 0

    for l in range(max(0, linha-1), min(self.tamanhoTabuleiro-1, linha+1)+1):
      for c in range(max(0, coluna-1), min(self.tamanhoTabuleiro-1, coluna+1)+1):
        if l == linha and c == coluna:
          # a posição inicial, não checamos.
          continue
        if self.tabuleiro[l][c] == "*":
          numeroDeBombasVizinhas += 1

    return numeroDeBombasVizinhas

  def escava(self, linha, coluna):
    self.escavado.add((linha, coluna))

    if self.tabuleiro[linha][coluna] == "*":
      return False
    elif self.tabuleiro[linha][coluna] > 0:
      return True

    for l in range(max(0, linha-1), min(self.tamanhoTabuleiro-1, linha+1)+1):
      for c in range(max(0, coluna-1), min(self.tamanhoTabuleiro-1, coluna+1)+1):
        if (l, c) in self.escavado:
          continue

        self.escava(l, c)

    return True

  def __str__(self):
    tabuleiroVisivel = [[None for _ in range(self.tamanhoTabuleiro)] for _ in range(self.tamanhoTabuleiro)]
    for linha in range(self.tamanhoTabuleiro):
      for coluna in range(self.tamanhoTabuleiro):
        if(linha, coluna) in self.escavado:
          tabuleiroVisivel[linha][coluna] = str(self.tabuleiro[linha][coluna])
        else:
          tabuleiroVisivel[linha][coluna] = " "

    string_rep = ""

    widths = []
    for idx in range(self.tamanhoTabuleiro):
      colunas = map(lambda x: x[idx], tabuleiroVisivel)
      widths.append(len(max(colunas, key = len)))

    indices = [i for i in range(self.tamanhoTabuleiro)]
    indices_linha = "   "
    celulas = []
    for idx, col in  enumerate(indices):
      format = "%-" + str(widths[idx]) + "s"
      celulas.append(format % (col))
    indices_linha += " ".join(celulas)
    indices_linha += " \n"

    for i in range(len(tabuleiroVisivel)):
      linha = tabuleiroVisivel[i]
      string_rep += f"{i} |"
      celulas = []
      for idx, col in enumerate(linha):
        format = "%-" + str(widths[idx]) + "s"
        celulas.append(format % (col))
      string_rep += " |".join(celulas)
      string_rep += " |\n"

    str_len = int(len(string_rep) / self.tamanhoTabuleiro)
    string_rep = indices_linha + "-"*str_len + "\n" + string_rep + "-"*str_len

    return string_rep


def jogar(tamanhoTabuleiro = 10, numeroBombas = 10):
  # 1° - criar o tabuleiro e colocar as bombas
  tabuleiro = Tabuleiro(tamanhoTabuleiro, numeroBombas)

  # 2° - mostrar o tabuleiro e perguntar onde o usuário quer cavar
  # 3°a - se o local é uma bomba, mostra que o jogo acabou
  # 3°b - se não é uma bomba, cavar recursivamente até toda posição ser no mínimo ao lado de uma bomba
  # 4° - repita o 2° e o 3°a/b passo até que não haja mais posições para procurar - Vitória!

  safe = True

  while len(tabuleiro.escavado) < tabuleiro.tamanhoTabuleiro ** 2 - numeroBombas:
    print(tabuleiro)
    inputUsuario = re.split(",(\\s)*", input("Onde você deseja cavar? Escreva 'linha, coluna': "))
    linha, coluna = int(inputUsuario[0]), int(inputUsuario[-1])
    if linha < 0 or linha >= tabuleiro.tamanhoTabuleiro or coluna < 0 or coluna >= tabuleiro.tamanhoTabuleiro:
      print("Posição inválida! Tente de novo.")
      continue

    safe = tabuleiro.escava(linha, coluna)
    if not safe:
      # escavou uma bomba :(
      break # game over

  if safe:
    print("Parabéns! Você venceu!!!")
  else:
    print("Você perdeu... :(")
    # vamos mostrar o tabuleiro
    tabuleiro.escavado = [(l, c) for l in range(tabuleiro.tamanhoTabuleiro) for c in range(tabuleiro.tamanhoTabuleiro)]
    print(tabuleiro)

if __name__ == '__main__':
  jogar()
